https://scryfall.com/docs/api/bulk-data

All Cards

In [1]:
import os
import winsound  # make sound when long cells have been executed

import orjson as json
from tqdm.auto import tqdm
from utils import NEW_DATA_FOLDER, SCRYFALL_FOLDER, create_directories, copy_old_data

create_directories()
copy_old_data()

Copied files


In [ ]:
# this will take time
with open(os.path.join(SCRYFALL_FOLDER, "all-cards.json"), encoding="utf-8") as j:
    data = json.loads(j.read())

# make a sound to let me know it finished
winsound.Beep(500, 400)

In [3]:
def clean_str(stringa: str) -> str:
    for elem in (" -", " /"):
        stringa = stringa.split(elem)[0]
    return stringa

In [4]:
card_types = dict()

errors = list()  # no italian name provided
translations = dict()

for dictionary in tqdm(data):
    if dictionary["lang"] == "en":
        lower_name = dictionary["name"].lower()
        # if card has 2 sides
        if "card_faces" in dictionary:
            # card type is first type
            card_types[lower_name] = clean_str(dictionary["card_faces"][0]["type_line"])
            for subdict in dictionary["card_faces"]:
                side_name = subdict["name"].lower()
                # some double faced tokens don't have type line for second face
                if "type_line" in subdict:
                    card_types[side_name] = clean_str(subdict["type_line"])
        elif dictionary["type_line"] != "Card":
            card_types[lower_name] = dictionary["type_line"]
    elif dictionary["lang"] == "it":
        try:
            translations[dictionary["printed_name"].lower()] = dictionary[
                "name"
            ].lower()
        except KeyError:
            ita_name = list()
            # split cards don't have "printed_name" but have "card_faces"
            if "card_faces" in dictionary:
                for subdict in dictionary["card_faces"]:
                    try:
                        ita_name.append(subdict["printed_name"].lower())
                        # old double cards have printed_name only for first half
                        # (e.g.: Domanda/Offerta)
                        if "/" in subdict["printed_name"]:
                            break
                    # Kamigawa split cards don't have printed_name for second face
                    except KeyError:
                        break
                ita_name = " // ".join(ita_name)
                translations[ita_name] = dictionary["name"].lower()
            else:
                errors.append(dictionary["name"])

errors = set(errors)
# del data  # save RAM by deleting reference

  0%|          | 0/383447 [00:00<?, ?it/s]

In [5]:
for d in data:
    if d["name"] in errors:
        display(d)
        break

{'object': 'card',
 'id': '00030770-5e99-4943-819d-8d807c24cc14',
 'oracle_id': '56719f6a-1a6c-4c0a-8d21-18f7d7350b68',
 'multiverse_ids': [489643],
 'arena_id': 72579,
 'tcgplayer_id': 215984,
 'cardmarket_id': 472559,
 'name': 'Swamp',
 'lang': 'en',
 'released_at': '2020-07-17',
 'uri': 'https://api.scryfall.com/cards/00030770-5e99-4943-819d-8d807c24cc14',
 'scryfall_uri': 'https://scryfall.com/card/jmp/59/swamp?utm_source=api',
 'layout': 'normal',
 'highres_image': True,
 'image_status': 'highres_scan',
 'image_uris': {'small': 'https://c1.scryfall.com/file/scryfall-cards/small/front/0/0/00030770-5e99-4943-819d-8d807c24cc14.jpg?1600716281',
  'normal': 'https://c1.scryfall.com/file/scryfall-cards/normal/front/0/0/00030770-5e99-4943-819d-8d807c24cc14.jpg?1600716281',
  'large': 'https://c1.scryfall.com/file/scryfall-cards/large/front/0/0/00030770-5e99-4943-819d-8d807c24cc14.jpg?1600716281',
  'png': 'https://c1.scryfall.com/file/scryfall-cards/png/front/0/0/00030770-5e99-4943-819d-

In [6]:
errors

{'Abyssal Horror',
 'Abyssal Specter',
 'Adarkar Wastes',
 'Aeon Chronicler',
 'Aether Flash',
 'Agonizing Memories',
 'Air Elemental',
 "Aladdin's Ring",
 'Anaconda',
 'Ancestral Memories',
 'Ancient Silverback',
 'Angelic Page',
 'Arcane Laboratory',
 'Archivist',
 'Ardent Militia',
 'Balduvian Barbarians',
 'Baleful Stare',
 'Beast of Burden',
 'Bedlam',
 'Befoul',
 'Bellowing Fiend',
 'Benthic Behemoth',
 'Bereavement',
 'Birds of Paradise',
 'Blanchwood Armor',
 'Blaze',
 'Blessed Reversal',
 'Blood Pet',
 'Bloodshot Cyclops',
 'Bog Imp',
 'Bog Wraith',
 'Boil',
 'Boomerang',
 'Breath of Life',
 'Brushland',
 'Bull Hippo',
 'Caltrops',
 'Canopy Spider',
 'Castle',
 'Catti-brie of Mithral Hall',
 'Charcoal Diamond',
 'Circle of Protection: Black',
 'Circle of Protection: Blue',
 'Circle of Protection: Green',
 'Circle of Protection: Red',
 'Circle of Protection: White',
 'City of Brass',
 'Cloudchaser Eagle',
 'Coat of Arms',
 'Compost',
 'Confiscate',
 'Coral Merfolk',
 'Corrupt',

In [7]:
print(len(errors))

348


### Corrections

In [8]:
corrections = {
    "prismari command": "Instant",
    "valentin, dean of the vein": "Creature",
    "arrogant poet": "Creature",
    "ephemerate": "Instant",
}

for k, v in corrections.items():
    if card_types[k] != v:
        # card_types[k] = v
        print(k, "is", card_types[k], "should be", v)

try:
    del card_types["lands"]
except KeyError:
    print("No lands")

valentin, dean of the vein is Legendary Creature — Vampire Warlock should be Creature
arrogant poet is Card should be Creature
No lands


In [9]:
print(card_types["liliana of the veil"])
# print(card_types['liliana del velo'])
print(card_types["lorehold command"])

Legendary Planeswalker — Liliana
Instant


In [10]:
print(card_types["clearwater pathway"])

Land


In [11]:
print(card_types["find // finality"])

Sorcery


In [12]:
print(translations["trovare // troncare"])

find // finality


### Create files

In [13]:
with open(
    os.path.join(NEW_DATA_FOLDER, "new_translations.py"), "w", encoding="utf-8"
) as f:
    f.write("translations = " + str(translations))